## Import Packages

In [1]:
import os
import mediapipe as mp
import cv2
import time

## Import Model, Configurations, adn Paths Initialization

In [2]:
model_path = 'model'
image_path = 'data'

# List Model & Image Paths
model_paths = [os.path.join(model_path, f) for f in os.listdir(model_path) if f.endswith('.tflite')]
image_paths = [os.path.join(image_path, f) for f in os.listdir(image_path) if f.lower().endswith(('.JPEG', '.jpeg'))]

# Ground Truth yang di-specify Manual
true_labels = {
    "ILSVRC2012_val_00000001.JPEG": "snake",
    "ILSVRC2012_val_00000002.JPEG": "ski",
    "ILSVRC2012_val_00000003.JPEG": "dog",
    "ILSVRC2012_val_00000004.JPEG": "bowl",
    "ILSVRC2012_val_00000005.JPEG": "bassinet",
    "ILSVRC2012_val_00000006.JPEG": "snake",
    "ILSVRC2012_val_00000007.JPEG": "porcupine",
    "ILSVRC2012_val_00000008.JPEG": "macaron",
    "ILSVRC2012_val_00000009.JPEG": "mouse",
    "ILSVRC2012_val_00000010.JPEG": "dog",
    "ILSVRC2012_val_00000011.JPEG": "coral",
    "ILSVRC2012_val_00000012.JPEG": "cougar",
    "ILSVRC2012_val_00000013.JPEG": "guenon",
    "ILSVRC2012_val_00000014.JPEG": "vehicle",
    "ILSVRC2012_val_00000015.JPEG": "harvester",
    "ILSVRC2012_val_00000016.JPEG": "whale",
    "ILSVRC2012_val_00000017.JPEG": "starfish",
    "ILSVRC2012_val_00000018.JPEG": "vulture",
    "ILSVRC2012_val_00000019.JPEG": "carton",
    "ILSVRC2012_val_00000020.JPEG": "crane",
    "ILSVRC2012_val_00000021.JPEG": "porcupine",
    "ILSVRC2012_val_00000022.JPEG": "hound",
    "ILSVRC2012_val_00000023.JPEG": "granny smith",
    "ILSVRC2012_val_00000024.JPEG": "planetarium",
    "ILSVRC2012_val_00000025.JPEG": "vulture",
    "ILSVRC2012_val_00000026.JPEG": "lamp",
    "ILSVRC2012_val_00000027.JPEG": "wolf",
    "ILSVRC2012_val_00000028.JPEG": "walker hound",
    "ILSVRC2012_val_00000029.JPEG": "snake",
    "ILSVRC2012_val_00000030.JPEG": "roof",
    "ILSVRC2012_val_00000031.JPEG": "butterfly",
    "ILSVRC2012_val_00000032.JPEG": "kart",
    "ILSVRC2012_val_00000033.JPEG": "otter",
    "ILSVRC2012_val_00000034.JPEG": "ballplayer",
    "ILSVRC2012_val_00000035.JPEG": "half track",
    "ILSVRC2012_val_00000036.JPEG": "vestment",
    "ILSVRC2012_val_00000037.JPEG": "common newt",
    "ILSVRC2012_val_00000038.JPEG": "abacus",
    "ILSVRC2012_val_00000039.JPEG": "scabbard",
    "ILSVRC2012_val_00000040.JPEG": "tick",
    "ILSVRC2012_val_00000041.JPEG": "bassinet",
    "ILSVRC2012_val_00000042.JPEG": "barrel",
    "ILSVRC2012_val_00000043.JPEG": "bill",
    "ILSVRC2012_val_00000044.JPEG": "schnauzer",
    "ILSVRC2012_val_00000045.JPEG": "newfoundland",
    "ILSVRC2012_val_00000046.JPEG": "pitcher",
    "ILSVRC2012_val_00000047.JPEG": "freight car",
    "ILSVRC2012_val_00000048.JPEG": "hound",
    "ILSVRC2012_val_00000049.JPEG": "cab",
    "ILSVRC2012_val_00000050.JPEG": "shop",
}

model_size = {
    "efficientnet_lite0.tflite": "5.3 MB",
    "efficientnet_lite0_float.tflite": "18.1 MB",
    "efficientnet_lite2.tflite": "6.9 MB",
    "efficientnet_lite2_float.tflite": "23.7 MB"
}

# Config Model
BaseOptions = mp.tasks.BaseOptions
ImageClassifier = mp.tasks.vision.ImageClassifier
ImageClassifierOptions = mp.tasks.vision.ImageClassifierOptions
VisionRunningMode = mp.tasks.vision.RunningMode

## Inference

In [4]:
for model in model_paths:
    print(f"\nModel {os.path.basename(model)}")
    # Load Model
    options = ImageClassifierOptions(
        base_options=BaseOptions(model_asset_path=model),
        max_results=3,
        running_mode=VisionRunningMode.IMAGE
    )

    # Variabel menyimpan latensi, top-1 acc, top-3 acc, dan confidence
    total_latency = 0
    top1_correct = 0
    top3_correct = 0
    matched_confidences = []

    with ImageClassifier.create_from_options(options) as classifier:
        for img_path in image_paths:
            filename = os.path.basename(img_path)

            # Load dan Convert Image
            img = cv2.imread(img_path)
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=img_rgb)

            # Menghitung Latensi & Melakukan Inference
            start_time = time.time()
            result = classifier.classify(mp_image)
            latency = (time.time() - start_time) * 1000
            total_latency += latency

            # Membandingkan dengan Ground Truth
            true_label = true_labels[filename].lower()
            categories = result.classifications[0].categories
            topk = [c.category_name.lower() for c in categories]
            topk_scores = [c.score for c in categories]

            # Top-1
            if true_label in topk[0]:
                top1_correct += 1
                matched_confidences.append(topk_scores[0])  # confidence of Top-1

            # Top-3 (Jika tidak match dengan top-1)
            else:
                for name, score in zip(topk[:3], topk_scores[:3]):
                    if true_label in name:
                        top3_correct += 1
                        matched_confidences.append(score)
                        break
            # Jika match saat Top-1, hitung match untuk Top-3
            if true_label in topk[0]:
                top3_correct += 1

    # Averaging semua metriks dengan jumlah image (confidence averaging dengan image yang berhasil ditebak)
    num_images = len(image_paths)
    avg_latency = total_latency / num_images
    top1_acc = top1_correct / num_images
    top3_acc = top3_correct / num_images
    avg_matched_conf = sum(matched_confidences) / len(matched_confidences) if matched_confidences else 0

    # Menampilkan Hasil Uji
    print(f"Model Size                  : {model_size[os.path.basename(model)]}")
    print(f"Average Latency             : {avg_latency:.2f} ms")
    print(f"Top-1 Accuracy              : {top1_acc:.2%}")
    print(f"Top-3 Accuracy              : {top3_acc:.2%}")
    print(f"Avg Confidence (Matched Top): {avg_matched_conf:.2f}")


Model efficientnet_lite0.tflite
Model Size                  : 5.3 MB
Average Latency             : 14.71 ms
Top-1 Accuracy              : 84.00%
Top-3 Accuracy              : 94.00%
Avg Confidence (Matched Top): 0.62

Model efficientnet_lite0_float.tflite
Model Size                  : 18.1 MB
Average Latency             : 23.16 ms
Top-1 Accuracy              : 88.00%
Top-3 Accuracy              : 94.00%
Avg Confidence (Matched Top): 0.66

Model efficientnet_lite2.tflite
Model Size                  : 6.9 MB
Average Latency             : 26.28 ms
Top-1 Accuracy              : 80.00%
Top-3 Accuracy              : 92.00%
Avg Confidence (Matched Top): 0.49

Model efficientnet_lite2_float.tflite
Model Size                  : 23.7 MB
Average Latency             : 48.76 ms
Top-1 Accuracy              : 84.00%
Top-3 Accuracy              : 94.00%
Avg Confidence (Matched Top): 0.52
